In [60]:
import pandas as pd
import numpy as np
os.chdir("/mnt/batch/tasks/shared/LS_root/mounts/clusters/dscpu/code/Users/Tamilselvan.S/Press Release/Data")
df = pd.read_excel("Press release domain.xlsx", engine = "openpyxl")

In [61]:
df = df[df["Language"] == "English"].reset_index(drop = True).copy()

In [62]:
df.head(2)

,Category,Date of release,Title / Contents,Issued by,Country,Language,Description,URL,Marketing Contact,E mail for Marketing Contact,Quarter,Unique Count,Company,Issued_Short,Domain
0,Corporate Information,2021-04-01,NTT DATA Ranked No. 8 in Brand Finance's Lates...,NTT DATA,Global,English,"TOKYO – Apr 1, 2021 – NTT DATA, a global digit...",https://www.nttdata.com/global/en/media/press-...,Amy Baj,amy.baj@nttdata.com,2021 Q1,NaN,NaN,Global (HQ),Financial Service
1,Corporate Information,2021-04-01,"Effective April 1, 2021: itelligence | NTT DAT...",NTT DATA,Global,English,"March 30, 2021\n itelligence | NTT DATA Busine...",https://us.nttdata.com/en/news/press-release/2...,Amy Baj,amy.baj@nttdata.com,2021 Q1,NaN,NaN,Global (HQ),Media


In [63]:
df["Sentence"] = df["Title / Contents"] + " " + df["Description"]

In [64]:
# !pip install -U sentence-transformers

from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [65]:
domain_list = ["Manufacturing", "Retail", "E-commerce", "Health", "Insurance", 
               "Financial Service", "Pharma", "Technology", "Energy",
              "Transport And Logistics", "Telecom", "Media", "Hospitality and Tourism",
               "Automobile", "Infrastructure", "Real estate","Defence","Government",
              ]

In [66]:
tensor_list = []
for i,k in enumerate(domain_list): 
#     print(k)
    tensor = model.encode(domain_list[i],convert_to_tensor=True)
    tensor_list.append(tensor)

    
label_data = pd.DataFrame()
label_data["Domain"] = domain_list
label_data["Domain_embed"] = tensor_list

##### Mechanism

In [69]:
# ###EXAMPLE 1


# sen = []

# sen_embed = model.encode(sen[0], convert_to_tensor=True)

# domain_names = []
# domain_scores = []
# for i,k in zip(label_data["Domain"], label_data["Domain_embed"]):
#     score = util.pytorch_cos_sim(sen_embed, k)
#     domain_names.append(i)
#     domain_scores.append(score)
# #     print(i, round(float(score),3))
    
# score_data = pd.DataFrame()
# score_data["Domain"] = domain_names
# score_data["Score"] = np.array(domain_scores)
# score_data.sort_values(by = "Score", ascending = False, inplace=True)
# score_data.reset_index(drop = True, inplace = True)
# list(score_data["Domain"].head(2))

In [70]:
def DomainClassifier(sen):
    sen_embed = model.encode(sen[0], convert_to_tensor=True)

    domain_names = []
    domain_scores = []
    
    for i,k in zip(label_data["Domain"], label_data["Domain_embed"]):
        score = util.pytorch_cos_sim(sen_embed, k)
        domain_names.append(i)
        domain_scores.append(score)

    score_data = pd.DataFrame()
    score_data["Domain"] = domain_names
    score_data["Score"] = np.array(domain_scores)
    score_data.sort_values(by = "Score", ascending = False, inplace=True)
    score_data.reset_index(drop = True, inplace = True)
    return list(score_data["Domain"].head(3))

In [71]:
result = df["Sentence"].apply(lambda sen:DomainClassifier([sen]))
df["Domain_Pred"] = result

In [72]:
result

0               [Technology, Financial Service, Telecom]
1               [Technology, Infrastructure, E-commerce]
2                      [Technology, Automobile, Telecom]
3               [Financial Service, Technology, Telecom]
4               [Technology, Financial Service, Telecom]
                             ...                        
502    [Automobile, Technology, Transport And Logistics]
503             [Financial Service, Technology, Telecom]
504             [Technology, Financial Service, Telecom]
505             [Financial Service, Technology, Telecom]
506             [Technology, Financial Service, Telecom]
Name: Sentence, Length: 507, dtype: object

In [73]:
df.to_csv("Domain_Pred_Data.csv", index = False)

In [9]:
###label similarity


sen = ["NTT DATA Ranked No. 8 in Brand Finance's Latest Report TOKYO – Apr 1, 2021 – NTT DATA, \
a global digital business and IT services leader, \
has been ranked the world's eighth most valuable brand among IT services providers, \
according to the Brand Finance IT Services 25 2021 report issued by UK-based Brand Finance, \
the world's leading brand valuation and strategy consultancy."]

sen_embed = model.encode(sen[0], convert_to_tensor=True)

print("Scores")
print()
for i,k in zip(label_data["Domain"], label_data["Domain_embed"]):
    score = util.pytorch_cos_sim(sen_embed, k)
    print(i, round(float(score),3))

Scores

Manufacturing 0.124
Retail 0.147
E-commerce 0.191
Health 0.127
Insurance 0.082
Financial Service 0.248
Pharma 0.135
Technology 0.248
Energy 0.045
Transport And Logistics 0.127
Telecom 0.238
Media 0.191
Hospitality and Tourism 0.056
Automobile 0.111
Infrastructure 0.135
Real estate 0.066
Defence 0.113
Government 0.097


In [89]:
###EXAMPLE 2


sen = [" publish Digital Thought Leadership in Automotive today published the white paper regarding \
Digital Thought Leadership in Automotive, which provides a brief on the technical capability of\
 within Automotive onboard and offboard areas through a technical overview."]

sen_embed = model.encode(sen[0], convert_to_tensor=True)

print("Scores")
print()
for i,k in zip(label_data["Domain"], label_data["Domain_embed"]):
    score = util.pytorch_cos_sim(sen_embed, k)
    print(i, round(float(score),3))

Scores

Manufacturing 0.207
Domain 0.136
Retail 0.147
E-commerce 0.141
Health 0.098
Insurance 0.051
Financial Service 0.165
Pharma 0.083
Technology 0.254
Energy 0.036
Transport And Logistics 0.207
Telecom 0.205
Media 0.199
Hospitality and Tourism 0.025


0.056

In [1]:
!pip install googletrans

     |████████████████████████████████| 55 kB 1.2 MB/s eta 0:00:011
     |████████████████████████████████| 58 kB 2.4 MB/s  eta 0:00:01
     |████████████████████████████████| 42 kB 288 kB/s  eta 0:00:01
     |████████████████████████████████| 1.5 MB 74.6 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 284 kB/s  eta 0:00:01
     |████████████████████████████████| 53 kB 888 kB/s  eta 0:00:01
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15735 sha256=eb8ba1500b78c19c0d2b785da81650cd9d44914c89308bb8478d754c35373e8f
  Stored in directory: /home/azureuser/.cache/pip/wheels/0e/ce/9b/d51de1064911d42480ab6b57fc943ee36572441f27546354e2
Successfully built googletrans
ERROR: ray 1.13.0 has requirement grpcio<=1.43.0,>=1.28.1, but you'll have grpcio 1.47.0 which is incompatible.
ERROR: pandas-profiling 3.2.0 has requirement joblib~=1.1.0, but you'll have joblib 0.14.1 which is incompatible.
ERROR: pandas-profiling 3.2.0 has requirement markupsafe~